In [1]:
data_dir="/home/ansary/work/hadith/hadith_data_bookwise"

In [ ]:
import os 
from glob import  glob
from tqdm import tqdm
import pandas as pd 
import json
tqdm.pandas()
from bs4 import BeautifulSoup
from collections import defaultdict
from itertools import  zip_longest
import re
from collections import defaultdict

def extract_teacher_student_relations(hadith_text):
    narrator_pattern = r'<a class=rawy id=(\d+)>(.*?)</a>'
    narrators = re.findall(narrator_pattern, hadith_text)

    matn_index = hadith_text.find('<a class=matn>')
    filtered_narrators = [n for n in narrators if hadith_text.find(n[1]) < matn_index]

    teacher_student_dict = defaultdict(lambda: {'teachers': [], 'students': []})

    for i in range(len(filtered_narrators) - 1):
        student_id, _ = filtered_narrators[i]
        teacher_id, _ = filtered_narrators[i + 1]

        teacher_student_dict[student_id]['teachers'].append(teacher_id)
        teacher_student_dict[teacher_id]['students'].append(student_id)

    return dict(teacher_student_dict)  # Convert defaultdict to normal dict


def build_teacher_student_dict(hadith_text, narrators):
    # Parse hadith text using BeautifulSoup to extract narrator IDs in order
    soup = BeautifulSoup(hadith_text, "html.parser")
    narrator_ids = [tag["id"] for tag in soup.find_all("a", class_="rawy")]
    
    # Create a dictionary for easy access to narrator details
    narrator_dict = {n["id"]: n for n in narrators}
    
    # Initialize teacher-student structure
    teacher_student = defaultdict(lambda: {"teachers": [], "students": []})
    
    # Build teacher-student relationships
    for i in range(len(narrator_ids) - 1):
        student_id = narrator_ids[i]
        teacher_id = narrator_ids[i + 1]
        
        # Add relationships
        teacher_student[teacher_id]["students"].append(student_id)
        teacher_student[student_id]["teachers"].append(teacher_id)
    
    return dict(teacher_student)

books=os.listdir(data_dir)


all_narrators=[]
all_books=[]
all_hadith_ids=[]
all_narrator_ids=[]
all_teacher_ids=[]
all_student_ids=[]

for book in books:
    book_dir=os.path.join(data_dir,book)
    hadiths=os.listdir(book_dir)
    for hadith in tqdm(hadiths):
        with open(os.path.join(book_dir,hadith),"r") as f:
            data=json.load(f)

        hadith_text=data['_source']['hadith']
        ta_dict=extract_teacher_student_relations(hadith_text)
        for nid in ta_dict.keys():
            for teacher,student in zip_longest(ta_dict[nid]["teachers"],ta_dict[nid]["students"]):
                all_books.append(book)
                all_hadith_ids.append(data["_source"]["hadith_id"])
                all_narrator_ids.append(nid)    
                all_teacher_ids.append(teacher)
                all_student_ids.append(student)
            

In [ ]:
df=pd.DataFrame({"book":all_books,
                 "hadith_id":all_hadith_ids,
                 "narrator_id":all_narrator_ids,
                 "teacher_id":all_teacher_ids,
                 "student_id":all_student_ids})
df.to_csv("student_teacher_data_bookwise.csv",index=False)

# Alternate chains

In [ ]:
import os 
from glob import  glob
from tqdm import tqdm
import pandas as pd 
import json
tqdm.pandas()
from bs4 import BeautifulSoup
from collections import defaultdict
from itertools import  zip_longest
import re
from collections import defaultdict

def extract_teacher_student_relations(hadith_text):
    narrator_pattern = r'<a class=rawy id=(\d+)>(.*?)</a>'
    narrators = re.findall(narrator_pattern, hadith_text)

    matn_index = hadith_text.find('<a class=matn>')
    filtered_narrators = [n for n in narrators if hadith_text.find(n[1]) > matn_index]

    teacher_student_dict = defaultdict(lambda: {'teachers': [], 'students': []})

    for i in range(len(filtered_narrators) - 1):
        student_id, _ = filtered_narrators[i]
        teacher_id, _ = filtered_narrators[i + 1]

        teacher_student_dict[student_id]['teachers'].append(teacher_id)
        teacher_student_dict[teacher_id]['students'].append(student_id)

    return dict(teacher_student_dict)  # Convert defaultdict to normal dict


books=os.listdir(data_dir)


all_narrators=[]
all_books=[]
all_hadith_ids=[]
all_narrator_ids=[]
all_teacher_ids=[]
all_student_ids=[]

for book in books:
    book_dir=os.path.join(data_dir,book)
    hadiths=os.listdir(book_dir)
    for hadith in tqdm(hadiths):
        with open(os.path.join(book_dir,hadith),"r") as f:
            data=json.load(f)

        hadith_text=data['_source']['hadith']
        ta_dict=extract_teacher_student_relations(hadith_text)
        for nid in ta_dict.keys():
            for teacher,student in zip_longest(ta_dict[nid]["teachers"],ta_dict[nid]["students"]):
                all_books.append(book)
                all_hadith_ids.append(data["_source"]["hadith_id"])
                all_narrator_ids.append(nid)    
                all_teacher_ids.append(teacher)
                all_student_ids.append(student)
            

In [ ]:
df=pd.DataFrame({"book":all_books,
                 "hadith_id":all_hadith_ids,
                 "narrator_id":all_narrator_ids,
                 "teacher_id":all_teacher_ids,
                 "student_id":all_student_ids})
df.to_csv("student_teacher_data_bookwise_alternate.csv",index=False)
df

In [ ]:
df.book.nunique()